# Analysing the cleaned Data

In [1]:
! git clone https://github.com/oke-aditya/Mental_Health_Analysis.git

Cloning into 'Mental_Health_Analysis'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 26 (delta 11), reused 11 (delta 4), pack-reused 0
Unpacking objects: 100% (26/26), done.


In [0]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [0]:
df = pd.read_csv("/content/Mental_Health_Analysis/cleaned_osmi.csv")

# Clean up Categorical Data

In [0]:
from sklearn import preprocessing
from pprint import pprint

In [5]:
df.head(2)

,*Are you self-employed?*,How many employees does your company or organization have?,Is your employer primarily a tech company/organization?,Is your primary role within your company related to tech/IT?,Does your employer provide mental health benefits as part of healthcare coverage?,Do you know the options for mental health care available under your employer-provided health coverage?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Does your employer offer resources to learn more about mental health disorders and options for seeking help?,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,"If a mental health issue prompted you to request a medical leave from work, how easy or difficult would it be to ask for that leave?",Would you feel more comfortable talking to your coworkers about your physical health or your mental health?,Would you feel comfortable discussing a mental health issue with your direct supervisor(s)?,Have you ever discussed your mental health with your employer?,Would you feel comfortable discussing a mental health issue with your coworkers?,Have you ever discussed your mental health with coworkers?,Have you ever had a coworker discuss their or another coworker's mental health with you?,"Overall, how much importance does your employer place on physical health?","Overall, how much importance does your employer place on mental health?",*Do you have previous employers?*,Was your employer primarily a tech company/organization?,Have your previous employers provided mental health benefits?,Were you aware of the options for mental health care provided by your previous employers?,Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?,Did your previous employers provide resources to learn more about mental health disorders and how to seek help?,Was your anonymity protected if you chose to take advantage of mental health or substance abuse treatment resources with previous employers?,Would you have felt more comfortable talking to your previous employer about your physical health or your mental health?,Would you have been willing to discuss your mental health with your direct supervisor(s)?,Did you ever discuss your mental health with your previous employer?,Would you have been willing to discuss your mental health with your coworkers at previous employers?,Did you ever discuss your mental health with a previous coworker(s)?,Did you ever have a previous coworker discuss their or another coworker's mental health with you?,"Overall, how much importance did your previous employer place on physical health?","Overall, how much importance did your previous employer place on mental health?",Do you *currently* have a mental health disorder?,Have you ever been *diagnosed* with a mental health disorder?,"*If so, what disorder(s) were you diagnosed with?*",Have you had a mental health disorder in the past?,Have you ever sought treatment for a mental health disorder from a mental health professional?,Do you have a family history of mental illness?,"If you have a mental health disorder, how often do you feel that it interferes with your work *when being treated effectively?*","If you have a mental health disorder, how often do you feel that it interferes with your work *when* _*NOT*_* being treated effectively (i.e., when you are experiencing symptoms)?*",Have your observations of how another individual who discussed a mental health issue made you less likely to reveal a mental health issue yourself in your current workplace?,How willing would you be to share with friends and family that you have a mental illness?,Would you be willing to bring up a physical health issue with a potential employer in an interview?,Why or why not?,Would you bring up your *mental* health with a potentia

In [0]:

def check_missing(column, check_df):
    if(check_df[column].isnull().any()):
        print("Number of rows having the NULL value in ",check_df," are ",check_df[column].isnull().sum())
    else:
        print("There is no NULL vale in the column", column)

def check_missing_all(check_df, verbose=False):
    empty_list = []
    for column in df.columns:
        if(check_df[column].isnull().any()):
            empty_rows = check_df[column].isnull().sum()
            empty_list.append(empty_rows)
            
            if(verbose == True):
                print("Number of rows having the NULL value in column %s are %d " %(column, empty_rows))
        else:
            print("There is no NULL value in the column %s" %(column))
            empty_list.append(0)
            
    return empty_list

def convert_columns(df):
    ''' Given a dataframe numbers the columns'''
    column_names = {}
    start_i = 1
    i = 1
    for column_name in df.columns:
        column_names[column_name] = str(i)
        i += 1
    
    df.rename(column_names, axis=1, inplace=True)
    print("Columns renamed strating with i = %d to i = %d" %(start_i,i))
    return df, column_names
#         print(column_name)

def clean_missing(df, column_list, empty_rows, percent_del=0.65):
    ''' Given a percentage it will remove columns that have data missing below that percentage.'''
    count = 0
    dropped_columns = []
    for i, column in enumerate(column_list):
        calc_percent = empty_rows[i] / len(df)
        if(calc_percent > percent_del):
            df.drop([column], axis=1, inplace=True)
            dropped_columns.append(column)
            print("%s Column dropped "%(column))
            count += 1
    
    print("Number of columns dropped %d"%(count))

    return df, dropped_columns

def encode_categorical(df, column_list):
    ''' Encodes categorical column using Ordinal Encoder '''
    for column in column_list:
        df[column] = df[column].astype('str')
        encoder = preprocessing.LabelEncoder()
        encoded_list = encoder.fit_transform(df[column])
#         print(encoded_list)
#         print(len(encoded_list))
        encoded_series = pd.Series(encoded_list)
        df[column] = encoded_series
        print("The ", column, "is encoded ")
        # Save the encoder so that we can do reverse transform
        # joblib.dump("{}_enocder.pkl".format(column), encoder)
    return(df)

# Numeric types need to do MinxMaxScaler
def scale_data(df, column_list, index_list):
    ''' Standardizes the data '''
    for column in column_list:
        df[column] = df[column].astype('float')
        encoder = preprocessing.StandardScaler()
        df[column] = encoder.fit_transform(df[column].values.reshape(-1,1))
        print("The ",column, "is encoded")
    return(df)


In [7]:
df, names = convert_columns(df)

Columns renamed strating with i = 1 to i = 62


In [8]:
df.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61
0,False,26-100,True,True,I don't know,No,Yes,Yes,I don't know,Very easy,Physical health,Yes,False,Yes,True,True,5.0,5.0,True,False,I don't know,N/A (was not aware),Some did,"Yes, they all did",I don't know,Physical health,Some of my previous supervisors,False,At some of my previous employers,False,True,8.0,5.0,Don't Know,NaN,NaN,No,False,No,Not applicable to me,Not applicable to me,No,8,Yes,If it impacts work I would bring it up,Maybe,If I feel it's under control I likely wouldn't...,False,8,No,Maybe/Not sure,4,NaN,False,25,Male,United States of America,Nebraska,White,United States of America,Nebraska
1,False,26-100,True,True,Yes,No,No,Yes,Yes,I don't know,Physical health,Maybe,False,Maybe,True,True,5.0,7.0,True,False,"Yes, they all did",I was aware of some,Some did,Some did,"Yes, always",Physical health,Some of my previous supervisors,False,At some of my previous employers,True,True,8.0,5.0,Possibly,NaN,NaN,Possibly,False,Yes,Sometimes,Often,No,7,No,I do not trust that the potential employer wou...,No,I do not trust that the potential employer wou...,False,7,"Yes, I observed","Yes, I experienced",1,NaN,False,51,male,United States of America,Nebraska,White,United States of America,Nebraska
2,False,26-100,True,True,I don't know,No,No,I don't know,I don't know,Somewhat difficult,Same level of comfort for each,No,False,Maybe,True,True,5.0,4.0,True,True,I don't know,N/A (was not aware),None did,None did,I don't know,Physical health,"No, none of my previous supervisors",False,"No, at none of my previous employers",False,False,10.0,2.0,No,NaN,NaN,No,False,I don't know,Not applicable to me,Not applicable to me,No,4,Maybe,It depends on its applicability to my job and ...,No,The stigma attached to mental health and not k...,False,4,Maybe/Not sure,"Yes, I observed",2,I think opening up more conversation around th...,True,27,Male,United States of America,Illinois,White,United States of America,Illinois
3,False,100-500,True,True,I don't know,No,Yes,Yes,Yes,Very easy,Physical health,Yes,False,Maybe,False,True,1.0,3.0,True,True,I don't know,N/A (was not aware),Some did,Some did,"Yes, always",Physical health,Some of my previous supervisors,False,At some of my previous employers,False,True,3.0,1.0,No,NaN,NaN,No,False,Yes,Not applicable to me,Not applicable to me,No,3,No,discrimination/bias,No,discrimination/bias,False,8,No,"Yes, I observed",3,NaN,False,37,male,United States of America,Nebraska,White,United States of America,Nebraska
4,False,26-100,True,True,I don't know,No,I don't know,I don't know,I don't know,I don't know,Physical health,No,False,No,False,False,4.0,4.0,True,True,I don't know,N/A (was not aware),None did,None did,I don't know,Physical health,"No, none of my previous supervisors",False,"No, at none of my previous employers",False,False,4.0,4.0,No,NaN,NaN,No,False,No,Not applicable to me,Not applicable to me,NaN,1,No,NaN,No,NaN,False,5,No,No,3,NaN,False,46,m,United States of America,Nebraska,White,United States of America,Nebraska


In [9]:
pprint(names)

{'*Are you self-employed?*': '1',
 '*Do you have previous employers?*': '19',
 '*If so, what disorder(s) were you diagnosed with?*': '36',
 'Are you openly identified at work as a person with a mental health issue?': '48',
 'Briefly describe what you think the industry as a whole and/or employers could do to improve mental health support for employees.': '53',
 'Did you ever discuss your mental health with a previous coworker(s)?': '30',
 'Did you ever discuss your mental health with your previous employer?': '28',
 "Did you ever have a previous coworker discuss their or another coworker's mental health with you?": '31',
 'Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?': '23',
 'Did your previous employers provide resources to learn more about mental health disorders and how to seek help?': '24',
 'Do you *currently* have a mental health disorder?': '34',
 'Do you have a family history of mental illness?

In [0]:
# drop some columns that are highly NLP based
df.drop(["35", "36", "45", "47", "53"], axis=1, inplace=True)

 'Have you ever sought treatment for a mental health disorder from a mental health professional?': '38',

 'Would you bring up your *mental* health with a potential employer in an interview?': '46',

In [11]:
categorical_columns = ['1', "2", '3','4','5','6','7','8', '9' ,'10', "11", "12", "13", "21", "22", "34", "37",
                       "14", "15", "16", "19", "20", "23", "24", "25", "26",  "27" , "28", "29",
                       "40", "41", "42", "44", "46", "39", "48", "56", "57", "58", "59", "60", "61",
                       "50", "51"] # just showing a sample of columns. We will extend this to all categorical columns.
df = encode_categorical(df, categorical_columns)

The  1 is encoded 
The  2 is encoded 
The  3 is encoded 
The  4 is encoded 
The  5 is encoded 
The  6 is encoded 
The  7 is encoded 
The  8 is encoded 
The  9 is encoded 
The  10 is encoded 
The  11 is encoded 
The  12 is encoded 
The  13 is encoded 
The  21 is encoded 
The  22 is encoded 
The  34 is encoded 
The  37 is encoded 
The  14 is encoded 
The  15 is encoded 
The  16 is encoded 
The  19 is encoded 
The  20 is encoded 
The  23 is encoded 
The  24 is encoded 
The  25 is encoded 
The  26 is encoded 
The  27 is encoded 
The  28 is encoded 
The  29 is encoded 
The  40 is encoded 
The  41 is encoded 
The  42 is encoded 
The  44 is encoded 
The  46 is encoded 
The  39 is encoded 
The  48 is encoded 
The  56 is encoded 
The  57 is encoded 
The  58 is encoded 
The  59 is encoded 
The  60 is encoded 
The  61 is encoded 
The  50 is encoded 
The  51 is encoded 


In [12]:
df.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,37,38,39,40,41,42,43,44,46,48,49,50,51,52,54,55,56,57,58,59,60,61
0,0,2,1,1,0,0,2,2,0,5,1,2,0,2,1,1,5.0,5.0,1,0,0,2,2,2,0,1,2,0,0,False,True,8.0,5.0,0,1,False,1,1,1,1,8,2,0,0,8,2,1,4,False,25,17,31,21,7,33,21
1,0,2,1,1,3,0,1,2,2,1,1,0,0,0,1,1,5.0,7.0,1,0,3,0,2,1,3,1,2,0,0,True,True,8.0,5.0,2,2,False,2,4,2,1,7,1,1,0,7,4,3,1,False,51,37,31,21,7,33,21
2,0,2,1,1,0,0,1,0,0,3,2,1,0,0,1,1,5.0,4.0,1,1,0,2,1,0,0,1,1,0,1,False,False,10.0,2.0,1,1,False,0,1,1,1,4,0,1,0,4,1,4,2,True,27,17,31,10,7,33,10
3,0,1,1,1,0,0,2,2,2,5,1,2,0,0,0,1,1.0,3.0,1,1,0,2,2,1,3,1,2,0,0,False,True,3.0,1.0,1,1,False,2,1,1,1,3,1,1,0,8,2,4,3,False,37,37,31,21,7,33,21
4,0,2,1,1,0,0,0,0,0,1,1,1,0,1,0,0,4.0,4.0,1,1,0,2,1,0,0,1,1,0,1,False,False,4.0,4.0,1,1,False,1,1,1,3,1,1,1,0,5,2,2,3,False,46,36,31,21,7,33,21


# Let us now try to predict a few questions

 'Have you ever sought treatment for a mental health disorder from a mental health professional?': '38',

 'Would you bring up your *mental* health with a potential employer in an interview?': '46',

- We wanted to optimize the sensitivity so let us use simple techniques such as logitic regression and decision trees first

In [0]:
# Given all the features has the person ever sougth for mental health disorder.

In [0]:
df.fillna(value=-1, inplace=True)

In [0]:
# Since column 38 was the above feature
y = df["38"] 
X = df.drop(["38"], axis=1)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [0]:
# Random forest classifier depends on Bagging 
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import naive_bayes

In [23]:
clf = RandomForestClassifier(max_depth=5, random_state=0)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [24]:
clf.score(X_train, y_train)

0.9572953736654805

We get decent accuracy without much feature engineering and simple data cleaning

In [28]:
# Slightly overfitting
clf.score(X_test, y_test)

0.8309859154929577

In [26]:
pprint(metrics.classification_report(y_test, clf.predict(X_test)))

('              precision    recall  f1-score   support\n'
 '\n'
 '       False       0.75      0.81      0.78        26\n'
 '        True       0.88      0.84      0.86        45\n'
 '\n'
 '    accuracy                           0.83        71\n'
 '   macro avg       0.82      0.83      0.82        71\n'
 'weighted avg       0.83      0.83      0.83        71\n')


In [46]:
# Boosting approach
clf = GradientBoostingClassifier(learning_rate=0.1, n_estimators=20, max_depth=5,)
clf.fit(X_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=5,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=20,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [47]:
# Overfits very easily
clf.score(X_train, y_train)

0.99644128113879

In [48]:
clf.score(X_test, y_test)

0.7464788732394366

In [49]:
pprint(metrics.classification_report(y_test, clf.predict(X_test)))

('              precision    recall  f1-score   support\n'
 '\n'
 '       False       0.64      0.69      0.67        26\n'
 '        True       0.81      0.78      0.80        45\n'
 '\n'
 '    accuracy                           0.75        71\n'
 '   macro avg       0.73      0.74      0.73        71\n'
 'weighted avg       0.75      0.75      0.75        71\n')


In [0]:
# Let us try with Naive Bayes as these columns look kinda independent

In [0]:
clf = naive_bayes.GaussianNB()

In [55]:
clf.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [57]:
clf.score(X_train, y_train)

0.8149466192170819

In [56]:
clf.score(X_test, y_test)

0.7887323943661971

In [58]:
pprint(metrics.classification_report(y_test, clf.predict(X_test)))

('              precision    recall  f1-score   support\n'
 '\n'
 '       False       0.70      0.73      0.72        26\n'
 '        True       0.84      0.82      0.83        45\n'
 '\n'
 '    accuracy                           0.79        71\n'
 '   macro avg       0.77      0.78      0.77        71\n'
 'weighted avg       0.79      0.79      0.79        71\n')
